In [1]:
import numpy as np
from pose_format import Pose
from pose_format.pose_visualizer import PoseVisualizer

## 2️⃣ تضخيم البيانات (Data Augmentation)

تضخيم البيانات مهم جداً لتدريب نماذج التعلم العميق

In [ ]:
def augment_pose_2d(pose: Pose, 
                    rotation_std: float = 0.2,
                    shear_std: float = 0.2,
                    scale_std: float = 0.2) -> Pose:
    """
    تضخيم 2D للـ pose
    
    Parameters:
    -----------
    rotation_std : float
        الانحراف المعياري للدوران (بالراديان)
    shear_std : float
        الانحراف المعياري للقص
    scale_std : float
        الانحراف المعياري للتحجيم
    """
    pose_copy = pose.copy()
    
    augmented = pose_copy.augment2d(
        rotation_std=rotation_std,
        shear_std=shear_std,
        scale_std=scale_std
    )
    
    print(f"✅ تم التضخيم 2D")
    print(f"   rotation_std={rotation_std}")
    print(f"   shear_std={shear_std}")
    print(f"   scale_std={scale_std}")
    
    return augmented

# augmented = augment_pose_2d(pose, rotation_std=0.1)

In [ ]:
def create_augmented_dataset(pose: Pose, num_augmentations: int = 10) -> list:
    """
    إنشاء مجموعة بيانات مضخمة
    """
    augmented_poses = [pose.copy()]  # الأصلي
    
    for i in range(num_augmentations):
        # تغيير معاملات التضخيم بشكل عشوائي
        rotation = np.random.uniform(0.05, 0.3)
        shear = np.random.uniform(0.05, 0.2)
        scale = np.random.uniform(0.05, 0.2)
        
        aug = pose.copy().augment2d(
            rotation_std=rotation,
            shear_std=shear,
            scale_std=scale
        )
        augmented_poses.append(aug)
    
    print(f"✅ تم إنشاء {len(augmented_poses)} نسخة (1 أصلية + {num_augmentations} مضخمة)")
    return augmented_poses

# augmented_dataset = create_augmented_dataset(pose, num_augmentations=5)

## 3️⃣ الاستيفاء وتغيير FPS

In [ ]:
def change_fps(pose: Pose, new_fps: float, kind: str = 'cubic') -> Pose:
    """
    تغيير معدل الإطارات باستخدام الاستيفاء
    
    Parameters:
    -----------
    new_fps : float
        معدل الإطارات الجديد
    kind : str
        نوع الاستيفاء: 'linear', 'quadratic', 'cubic'
    """
    pose_copy = pose.copy()
    
    original_fps = pose_copy.body.fps
    original_frames = pose_copy.body.data.shape[0]
    
    interpolated = pose_copy.interpolate(new_fps=new_fps, kind=kind)
    
    new_frames = interpolated.body.data.shape[0]
    
    print(f"✅ تم تغيير FPS")
    print(f"   من: {original_fps} FPS ({original_frames} إطار)")
    print(f"   إلى: {new_fps} FPS ({new_frames} إطار)")
    print(f"   نوع الاستيفاء: {kind}")
    
    return interpolated

# مثال: تحويل من 30 FPS إلى 24 FPS
# pose_24fps = change_fps(pose, new_fps=24)

# مثال: تحويل إلى 60 FPS (upsampling)
# pose_60fps = change_fps(pose, new_fps=60, kind='cubic')

In [ ]:
def interpolate_missing_frames(pose: Pose) -> Pose:
    """
    ملء الإطارات المفقودة بالاستيفاء
    (يحافظ على نفس FPS)
    """
    pose_copy = pose.copy()
    
    # الاستيفاء بنفس FPS يملأ القيم المفقودة
    interpolated = pose_copy.interpolate(new_fps=None, kind='linear')
    
    print("✅ تم ملء الإطارات المفقودة")
    return interpolated

# pose_filled = interpolate_missing_frames(pose)

## 4️⃣ قص وتحديد مكونات

In [ ]:
def get_specific_components(pose: Pose, components: list) -> Pose:
    """
    استخراج مكونات محددة
    
    Parameters:
    -----------
    components : list
        قائمة أسماء المكونات
    """
    filtered = pose.get_components(components)
    
    print(f"✅ تم استخراج المكونات: {components}")
    print(f"   عدد النقاط الأصلي: {pose.body.data.shape[2]}")
    print(f"   عدد النقاط الجديد: {filtered.body.data.shape[2]}")
    
    return filtered

# مثال: اليدين فقط
# hands_only = get_specific_components(pose, ['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS'])

# مثال: الجسم والوجه
# body_face = get_specific_components(pose, ['POSE_LANDMARKS', 'FACE_LANDMARKS'])

In [ ]:
def get_specific_points(pose: Pose, components: list, points_dict: dict) -> Pose:
    """
    استخراج نقاط محددة من مكونات محددة
    
    Parameters:
    -----------
    components : list
        قائمة أسماء المكونات
    points_dict : dict
        قاموس: اسم المكون -> قائمة أسماء النقاط
    """
    filtered = pose.get_components(components, points=points_dict)
    
    print(f"✅ تم استخراج النقاط المحددة")
    return filtered

# مثال: فقط نقاط معينة من الجسم
# upper_body = get_specific_points(
#     pose,
#     ['POSE_LANDMARKS'],
#     {
#         'POSE_LANDMARKS': [
#             'NOSE', 'LEFT_SHOULDER', 'RIGHT_SHOULDER',
#             'LEFT_ELBOW', 'RIGHT_ELBOW',
#             'LEFT_WRIST', 'RIGHT_WRIST'
#         ]
#     }
# )

In [ ]:
def remove_components(pose: Pose, components_to_remove: list) -> Pose:
    """
    إزالة مكونات من الـ pose
    """
    filtered = pose.remove_components(components_to_remove)
    
    print(f"✅ تمت إزالة المكونات: {components_to_remove}")
    return filtered

# مثال: إزالة POSE_WORLD_LANDMARKS
# pose_no_world = remove_components(pose, ['POSE_WORLD_LANDMARKS'])

## 5️⃣ حساب Bounding Box

In [ ]:
def compute_bounding_box(pose: Pose) -> Pose:
    """
    حساب bounding box لكل مكون
    
    يعطي نقطتين لكل مكون:
    - TOP_LEFT: الزاوية العليا اليسرى
    - BOTTOM_RIGHT: الزاوية السفلى اليمنى
    """
    bbox_pose = pose.bbox()
    
    print("✅ تم حساب Bounding Box")
    print(f"   الشكل الأصلي: {pose.body.data.shape}")
    print(f"   شكل bbox: {bbox_pose.body.data.shape}")
    
    return bbox_pose

# bbox = compute_bounding_box(pose)

In [ ]:
def get_hands_bounding_box(pose: Pose) -> dict:
    """
    الحصول على bounding box لليدين فقط
    """
    # استخراج اليدين
    hands = pose.get_components(['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS'])
    
    # حساب bbox
    bbox = hands.bbox()
    
    # استخراج الإحداثيات
    data = bbox.body.data
    
    result = {}
    for frame_idx in range(data.shape[0]):
        # اليد اليسرى: النقاط 0-1, اليد اليمنى: النقاط 2-3
        left_tl = data[frame_idx, 0, 0, :2]  # Top-Left
        left_br = data[frame_idx, 0, 1, :2]  # Bottom-Right
        right_tl = data[frame_idx, 0, 2, :2]
        right_br = data[frame_idx, 0, 3, :2]
        
        result[frame_idx] = {
            'left_hand': {'top_left': left_tl, 'bottom_right': left_br},
            'right_hand': {'top_left': right_tl, 'bottom_right': right_br}
        }
    
    return result

# boxes = get_hands_bounding_box(pose)

## 6️⃣ تسقيط الإطارات (Frame Dropout)

In [ ]:
def frame_dropout_uniform(pose: Pose, 
                          dropout_min: float = 0.2,
                          dropout_max: float = 1.0) -> tuple:
    """
    تسقيط إطارات بشكل منتظم
    
    Parameters:
    -----------
    dropout_min : float
        الحد الأدنى لنسبة الإبقاء
    dropout_max : float
        الحد الأقصى لنسبة الإبقاء
    
    Returns:
    --------
    tuple : (pose جديد, قائمة الإطارات المختارة)
    """
    dropped_pose, selected_indices = pose.frame_dropout_uniform(
        dropout_min=dropout_min,
        dropout_max=dropout_max
    )
    
    print(f"✅ تم تسقيط الإطارات (Uniform)")
    print(f"   الإطارات الأصلية: {pose.body.data.shape[0]}")
    print(f"   الإطارات المتبقية: {dropped_pose.body.data.shape[0]}")
    print(f"   نسبة الإبقاء: {len(selected_indices)/pose.body.data.shape[0]:.2%}")
    
    return dropped_pose, selected_indices

# dropped, indices = frame_dropout_uniform(pose, dropout_min=0.5, dropout_max=0.8)

In [ ]:
def frame_dropout_normal(pose: Pose,
                         dropout_mean: float = 0.5,
                         dropout_std: float = 0.1) -> tuple:
    """
    تسقيط إطارات بتوزيع طبيعي
    
    Parameters:
    -----------
    dropout_mean : float
        متوسط نسبة الإبقاء
    dropout_std : float
        الانحراف المعياري
    """
    dropped_pose, selected_indices = pose.frame_dropout_normal(
        dropout_mean=dropout_mean,
        dropout_std=dropout_std
    )
    
    print(f"✅ تم تسقيط الإطارات (Normal)")
    print(f"   الإطارات الأصلية: {pose.body.data.shape[0]}")
    print(f"   الإطارات المتبقية: {dropped_pose.body.data.shape[0]}")
    
    return dropped_pose, selected_indices

# dropped, indices = frame_dropout_normal(pose)

## 7️⃣ قلب الـ Pose (Flip)

In [ ]:
def flip_pose(pose: Pose, axis: int = 0) -> Pose:
    """
    قلب الـ pose على محور معين
    
    Parameters:
    -----------
    axis : int
        0 = قلب أفقي (X)
        1 = قلب رأسي (Y)
        2 = قلب العمق (Z)
    """
    flipped = pose.flip(axis=axis)
    
    axis_names = {0: 'X (أفقي)', 1: 'Y (رأسي)', 2: 'Z (عمق)'}
    print(f"✅ تم قلب الـ pose على المحور {axis_names.get(axis, axis)}")
    
    return flipped

# قلب أفقي (مثل المرآة)
# flipped = flip_pose(pose, axis=0)

In [ ]:
def mirror_pose_for_augmentation(pose: Pose) -> Pose:
    """
    قلب الـ pose أفقياً (للتضخيم)
    مفيد لمضاعفة بيانات التدريب
    """
    mirrored = pose.flip(axis=0)
    print("✅ تم إنشاء نسخة معكوسة")
    return mirrored

# mirrored = mirror_pose_for_augmentation(pose)

## 8️⃣ التحويل بين Backends

In [ ]:
def convert_to_torch(pose: Pose):
    """
    تحويل إلى PyTorch tensors
    """
    torch_pose = pose.torch()
    
    print(f"✅ تم التحويل إلى PyTorch")
    print(f"   نوع البيانات: {type(torch_pose.body.data)}")
    
    return torch_pose

# torch_pose = convert_to_torch(pose)

In [ ]:
def convert_to_tensorflow(pose: Pose):
    """
    تحويل إلى TensorFlow tensors
    """
    tf_pose = pose.tensorflow()
    
    print(f"✅ تم التحويل إلى TensorFlow")
    print(f"   نوع البيانات: {type(tf_pose.body.data)}")
    
    return tf_pose

# tf_pose = convert_to_tensorflow(pose)

## 9️⃣ دوال مساعدة إضافية

In [ ]:
def focus_pose(pose: Pose) -> Pose:
    """
    ضبط الـ pose ليبدأ من (0,0) ويملأ المساحة المتاحة
    مفيد للعرض والتصوير
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    print("✅ تم ضبط الـ pose (focus)")
    return pose_copy

# focused = focus_pose(pose)

In [ ]:
def slice_pose(pose: Pose, start: int = 0, end: int = None, step: int = 1) -> Pose:
    """
    قص جزء من الـ pose
    
    Parameters:
    -----------
    start : int
        إطار البداية
    end : int
        إطار النهاية (None = آخر إطار)
    step : int
        الخطوة (1 = كل إطار، 2 = كل إطارين، ...)
    """
    if end is None:
        end = pose.body.data.shape[0]
    
    sliced = pose.slice_step(start=start, end=end, step=step)
    
    print(f"✅ تم قص الـ pose")
    print(f"   من الإطار {start} إلى {end} بخطوة {step}")
    print(f"   الإطارات: {pose.body.data.shape[0]} -> {sliced.body.data.shape[0]}")
    
    return sliced

# أول 100 إطار
# first_100 = slice_pose(pose, start=0, end=100)

# كل إطار ثاني
# every_second = slice_pose(pose, step=2)

In [ ]:
def flatten_pose_data(pose: Pose) -> np.ndarray:
    """
    تسطيح البيانات لاستخدامها في ML
    يحذف النقاط ذات الثقة صفر
    """
    flat = pose.body.flatten()
    
    print(f"✅ تم تسطيح البيانات")
    print(f"   الشكل: {flat.shape}")
    print(f"   الأعمدة: [frame, person, point, confidence, x, y, z]")
    
    return flat

# flat_data = flatten_pose_data(pose)

## 🔟 مثال شامل: خط معالجة كامل

In [ ]:
def complete_preprocessing_pipeline(pose_path: str, output_path: str):
    """
    خط معالجة كامل للتحضير للتدريب
    """
    print("=" * 60)
    print("🚀 بدء خط المعالجة")
    print("=" * 60)
    
    # 1. تحميل
    print("\n1️⃣ تحميل الملف...")
    pose = load_pose(pose_path)
    print(f"   الإطارات: {pose.body.data.shape[0]}")
    
    # 2. استخراج المكونات المهمة فقط
    print("\n2️⃣ استخراج الجسم واليدين...")
    pose = pose.get_components([
        'POSE_LANDMARKS',
        'LEFT_HAND_LANDMARKS',
        'RIGHT_HAND_LANDMARKS'
    ])
    
    # 3. ملء الإطارات المفقودة
    print("\n3️⃣ ملء الإطارات المفقودة...")
    pose = pose.interpolate(new_fps=None, kind='linear')
    
    # 4. التطبيع
    print("\n4️⃣ التطبيع...")
    pose.normalize()
    
    # 5. تغيير FPS إلى قيمة ثابتة
    print("\n5️⃣ توحيد FPS إلى 25...")
    pose = pose.interpolate(new_fps=25, kind='cubic')
    
    # 6. حفظ
    print(f"\n6️⃣ حفظ في {output_path}...")
    with open(output_path, 'wb') as f:
        pose.write(f)
    
    print("\n" + "=" * 60)
    print("✅ تمت المعالجة بنجاح!")
    print(f"   الإطارات النهائية: {pose.body.data.shape[0]}")
    print(f"   النقاط: {pose.body.data.shape[2]}")
    print("=" * 60)
    
    return pose

# processed = complete_preprocessing_pipeline("input.pose", "processed.pose")

## 📚 ملخص الميزات

| الميزة | الدالة | الاستخدام |
|--------|--------|----------|
| التطبيع | `pose.normalize()` | توحيد الحجم والموقع |
| التطبيع الإحصائي | `pose.normalize_distribution()` | Mean=0, Std=1 |
| التضخيم | `pose.augment2d()` | زيادة بيانات التدريب |
| تغيير FPS | `pose.interpolate()` | توحيد معدل الإطارات |
| استخراج مكونات | `pose.get_components()` | اليدين فقط، الوجه، إلخ |
| إزالة مكونات | `pose.remove_components()` | حذف غير الضروري |
| Bounding Box | `pose.bbox()` | حساب حدود المكونات |
| تسقيط الإطارات | `pose.frame_dropout_*()` | تضخيم بحذف |
| القلب | `pose.flip()` | انعكاس أفقي/رأسي |
| التحويل | `pose.torch()` / `pose.tensorflow()` | للتدريب |

### نصائح للتدريب
1. **طبّع دائماً** قبل التدريب
2. **وحّد FPS** لكل البيانات
3. **استخدم التضخيم** لزيادة البيانات
4. **احذف المكونات غير الضرورية** لتقليل الأبعاد